## Energy Savings App

The Energy Saving Application is a comprehensive, modular pipeline designed to train and deploy a Reinforcement Learning (RL) agent for dynamic network optimization. The core objective is to significantly reduce power consumption by intelligently turning cell sectors on or off and adjusting their antenna tilts based on the time of day, while maintaining acceptable network coverage and Quality of Service (QoS).

This system leverages a pre-trained Bayesian Digital Twin (BDT) RF model, which allows the RL training loop to perform rapid, local RF simulations. This architecture decouples the RL agent's learning process from the latency of live backend services, enabling efficient training on multi-day traffic patterns. The entire workflow is orchestrated through a main application with a clear command-line interface.

In [ ]:
import sys
from pathlib import Path
sys.path.append(f"{Path().absolute().parent}")

In [ ]:
import logging
import os

from apps.energy_saving_app.data_preprocessor import UEDataPreprocessor
from apps.energy_saving_app.bdt_manager import BDTManager
from apps.energy_saving_app.rl_trainer import run_rl_training

In [ ]:
NOTEBOOK_DIR = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, ".."))

In [ ]:
# --- Logging Setup ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

### Setting up Data Directory

In [ ]:
BASE_DATA_DIR = os.path.join(NOTEBOOK_DIR, "data", "energy_saving_data/generated_data")
STATIC_DATA_DIR = os.path.join(NOTEBOOK_DIR, "data", "energy_saving_data/data")

TOPOLOGY_PATH = os.path.join(STATIC_DATA_DIR, "topology.csv")
CONFIG_PATH = os.path.join(STATIC_DATA_DIR, "config.json")
TRAINING_DATA_PATH = os.path.join(STATIC_DATA_DIR, "dummy_ue_training_data.csv")
BDT_MODEL_PATH = os.path.join("data", "energy_saving_data", "bdt_model_map.pickle")

RL_MODEL_PATH = os.path.join("data", "energy_saver_agent.zip")
RL_LOG_DIR = os.path.join("data", "rl_training_logs")

### Define the training and test days

In [ ]:
TRAIN_DAYS = [0, 1, 2, 3]
test_day = 4
days_to_process = sorted(set(TRAIN_DAYS + [test_day]))
TOTAL_TIMESTEPS = 10000

### Preprocessing

In [ ]:
# Run preprocessing
logger.info(f"--- Running UE Data Preprocessing for days: {days_to_process} ---")
preprocessor = UEDataPreprocessor(base_data_dir=BASE_DATA_DIR)
preprocessor.run(days=days_to_process)
logger.info("--- Preprocessing Step Finished ---")

### Training the core BDT Model

In [ ]:
# Parameters
BDT_MODEL_ID = "bdt_for_energy_saving"
CONTAINER_NAME = "radp_dev-training-1"

In [ ]:
# BDT Training
logger.info("--- Running BDT Training ---")
bdt_manager = BDTManager(
    topology_path=TOPOLOGY_PATH,
    training_data_path=TRAINING_DATA_PATH,
    model_path=BDT_MODEL_PATH
)
bdt_manager.train(model_id=BDT_MODEL_ID, container_name=CONTAINER_NAME)
logger.info("--- BDT Training Finished ---")

### Train RL Agent on The First 4 Days

In [ ]:
logger.info(f"--- Running RL Training on Days: {TRAIN_DAYS} ---")
run_rl_training(
    bdt_model_path=BDT_MODEL_PATH,
    base_ue_data_dir=BASE_DATA_DIR,
    training_days=TRAIN_DAYS,
    topology_path=TOPOLOGY_PATH,
    config_path=CONFIG_PATH,
    rl_model_path=RL_MODEL_PATH,
    log_dir=RL_LOG_DIR,
    total_timesteps=TOTAL_TIMESTEPS
)
logger.info("--- RL Training Finished ---")